In [ ]:
# 🔧 그룹 1: 설정 + 함수 정의
# (이 셀을 실행하면 모든 함수가 정의됩니다. 크롤링은 시작되지 않음)

import pandas as pd
import warnings, os, time, shutil, urllib, random
warnings.filterwarnings(action='ignore')

import re                        # 가격/평점 정제용 정규식
import json                      # 메타데이터 JSON 저장용  
from datetime import datetime    # 타임스탬프용

from PIL import Image
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, WebDriverException

import chromedriver_autoinstaller
import undetected_chromedriver as uc
from user_agents import parse
import selenium

print(f"🔧 Selenium 버전: {selenium.__version__}")

# ⭐⭐⭐ 중요 설정: 여기서 수정하세요! ⭐⭐⭐
CONFIG = {
    "WAIT_TIMEOUT": 10,
    "RETRY_COUNT": 3,
    "MIN_DELAY": 3,
    "MAX_DELAY": 8,
    "POPUP_WAIT": 5,
    "USER_AGENT": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/136.0.0.0 Whale/4.32.315.22 Safari/537.36",
    "SAVE_IMAGES": True,
    "SAVE_INTERMEDIATE": True,
    "MAX_PRODUCT_NAME_LENGTH": 30,
    
    # 🔢 도시당 크롤링할 상품 개수 설정 (여기서 변경!)
    "MAX_PRODUCTS_PER_CITY": 2,
}

# 🏙️ 검색할 도시들 (여기서 변경!)
CITIES_TO_SEARCH = ["오사카"]

# 도시별 대륙 및 국가 정보 매핑
CITY_INFO = {
    "방콕": {"대륙": "아시아", "국가": "태국"},
    "도쿄": {"대륙": "아시아", "국가": "일본"},
    "오사카": {"대륙": "아시아", "국가": "일본"},
    "싱가포르": {"대륙": "아시아", "국가": "싱가포르"},
    "홍콩": {"대륙": "아시아", "국가": "홍콩"},
    "파리": {"대륙": "유럽", "국가": "프랑스"},
    "런던": {"대륙": "유럽", "국가": "영국"},
    "뉴욕": {"대륙": "북미", "국가": "미국"},
    "시드니": {"대륙": "오세아니아", "국가": "호주"},
}
# 기존 CITY_INFO 코드는 그대로 두고...

# 🆕 여기에 추가!
CITY_CODES = {
    # 동남아시아
    "방콕": "BKK",
    "치앙마이": "CNX", 
    "푸켓": "HKT",
    "싱가포르": "SIN",
    "홍콩": "HKG",
    "쿠알라룸푸르": "KUL",
    "세부": "CEB",
    "다낭": "DAD",
    "호치민": "SGN",
    
    # 일본
    "도쿄": "NRT",
    "오사카": "KIX",
    "나고야": "NGO",
    "후쿠오카": "FUK",
    "오키나와": "OKA",
    "삿포로": "CTS",
    
    # 한국
    "서울": "ICN",
    "부산": "PUS",
    "제주": "CJU",
    "대구": "TAE",
    "광주": "KWJ",
    "여수": "RSU",
    
    # 유럽
    "파리": "CDG",
    "런던": "LHR",
    "로마": "FCO",
    "바르셀로나": "BCN",
    
    # 북미
    "뉴욕": "JFK",
    "로스앤젤레스": "LAX",
    "시카고": "ORD",
    
    # 오세아니아
    "시드니": "SYD",
    "멜버른": "MEL",
}

print(f"✅ CITY_CODES 추가 완료! {len(CITY_CODES)}개 도시 지원")

# ===== 모든 함수들 정의 =====

def print_progress(current, total, city_name, status="진행중"):
    """진행률을 시각적으로 표시하는 함수"""
    percentage = (current / total) * 100
    bar_length = 30
    filled_length = int(bar_length * current // total)
    bar = '█' * filled_length + '░' * (bar_length - filled_length)
    
    emoji = "🔍" if status == "진행중" else "✅" if status == "완료" else "❌"
    
    print(f"\n{emoji} 진행률: [{bar}] {percentage:.1f}% ({current}/{total})")
    print(f"📍 현재 작업: {city_name} - {status}")
    print("-" * 50)

def print_product_progress(current, total, product_name):
    """상품별 진행률 표시 함수"""
    percentage = (current / total) * 100
    bar_length = 20
    filled_length = int(bar_length * current // total)
    bar = '█' * filled_length + '░' * (bar_length - filled_length)
    
    safe_name = str(product_name)[:30] + "..." if len(str(product_name)) > 30 else str(product_name)
    print(f"    🎯 상품 진행률: [{bar}] {percentage:.1f}% ({current}/{total})")
    print(f"    📦 현재 상품: {safe_name}")

def save_intermediate_results(results, city_name):
    """중간 결과를 임시 파일로 저장하는 함수"""
    if results and CONFIG["SAVE_INTERMEDIATE"]:
        try:
            timestamp = time.strftime('%Y%m%d_%H%M%S')
            temp_filename = f"temp_중간저장_{city_name}_{timestamp}.csv"
            pd.DataFrame(results).to_csv(temp_filename, index=False, encoding='utf-8-sig')
            print(f"  💾 중간 결과 저장: {temp_filename}")
            return temp_filename
        except Exception as e:
            print(f"  ⚠️ 중간 저장 실패: {e}")
            return None
    return None

def retry_operation(func, operation_name, max_retries=None):
    """실패한 작업을 재시도하는 함수"""
    if max_retries is None:
        max_retries = CONFIG["RETRY_COUNT"]
    
    for attempt in range(max_retries):
        try:
            return func()
        except (TimeoutException, NoSuchElementException, WebDriverException) as e:
            if attempt == max_retries - 1:
                print(f"  ❌ {operation_name} 최종 실패: {type(e).__name__}")
                raise e
            print(f"  🔄 {operation_name} 재시도 {attempt + 1}/{max_retries} (오류: {type(e).__name__})")
            time.sleep(2)
        except Exception as e:
            print(f"  ❌ {operation_name} 예상치 못한 오류: {type(e).__name__}: {e}")
            raise e

def get_city_info(city_name):
    """도시명으로 대륙과 국가 정보를 가져오는 함수"""
    info = CITY_INFO.get(city_name, {"대륙": "기타", "국가": "기타"})
    return info["대륙"], info["국가"]

def make_safe_filename(filename):
    """파일명에 사용할 수 없는 문자 제거"""
    if not filename:
        return "기본파일명"
    
    safe_filename = str(filename)
    unsafe_chars = ['/', '\\', ':', '*', '?', '"', '<', '>', '|', '\n', '\r', '\t']
    for char in unsafe_chars:
        safe_filename = safe_filename.replace(char, '_')
    
    if len(safe_filename) > 200:
        safe_filename = safe_filename[:200]
    
    if safe_filename.startswith('.'):
        safe_filename = '_' + safe_filename[1:]
    
    return safe_filename

def make_user_agent(ua, is_mobile):
    user_agent = parse(ua)
    model = user_agent.device.model
    platform = user_agent.os.family
    platform_version = user_agent.os.version_string + ".0.0"
    version = user_agent.browser.version[0]
    ua_full_version = user_agent.browser.version_string
    architecture = "x86"
    print(platform)
    if is_mobile:
        platform_info = "Linux armv8l"
        architecture= ""
    else:
        platform_info = "Win32"
        model = ""
    RET_USER_AGENT = {
        "appVersion" : ua.replace("Mozilla/", ""),
        "userAgent": ua,
        "platform" : f"{platform_info}",
        "acceptLanguage" : "ko-KR, kr, en-US, en",
        "userAgentMetadata":{
            "brands" : [
                {"brand":"Google Chrome", "version":f"{version}"},
                {"brand":"Chromium", "version":f"{version}"},
                {"brand":" Not A;Brand", "version":"99"}
            ],
            "fullVersionList" : [
                {"brand":"Google Chrome", "version":f"{version}"},
                {"brand":"Chromium", "version":f"{version}"},
                {"brand":" Not A;Brand", "version":"99"}
            ],
            "fullVersion":f"{ua_full_version}",
            "platform" :platform,
            "platformVersion":platform_version,
            "architecture":architecture,
            "model" : model,
            "mobile":is_mobile
        }
    }
    return RET_USER_AGENT

def generate_random_geolocation():
    ltop_lat = 37.75415601640249
    ltop_long = 126.86767642302573
    rbottom_lat = 37.593829172663945
    rbottom_long = 127.15276051439332

    targetLat = random.uniform(rbottom_lat, ltop_lat)
    targetLong = random.uniform(ltop_long,rbottom_long)
    return {"latitude":targetLat, "longitude" : targetLong, "accuracy":100}

def setup_driver():
    """크롬 드라이버 설정 및 실행"""
    chromedriver_autoinstaller.install()
    
    options = uc.ChromeOptions()
    
    UA = CONFIG["USER_AGENT"]
    options.add_argument(f"--user-agent={UA}")
    
    rand_user_folder = random.randrange(1,100)
    raw_path = os.path.abspath("cookies")
    try:
        shutil.rmtree(raw_path)
    except:
        pass
    os.makedirs(raw_path, exist_ok=True)
    user_cookie_name = f"{raw_path}/{rand_user_folder}"
    if os.path.exists(user_cookie_name) == False:
        os.makedirs(user_cookie_name, exist_ok=True)
    
    try:
        driver = uc.Chrome(user_data_dir=user_cookie_name, options=options)
        print("✅ 크롬 드라이버 실행 성공!")
    except Exception as e:
        print('\n',"-"*50,"\n","-"*50,"\n")
        print("# 키홈 메세지 : 혹시 여기서 에러 발생시 [아래 블로그 참고 -> 재부팅 -> 다시 코드실행] 해보시길 바랍니다! \n (구글크롬 버젼 업그레이드 문제)")
        print('https://appfollow.tistory.com/102')
        print('\n',"-"*50,"\n","-"*50,"\n")
        raise RuntimeError
        
    UA_Data = make_user_agent(UA,False)
    driver.execute_cdp_cmd("Network.setUserAgentOverride",UA_Data)
    
    GEO_DATA = generate_random_geolocation()
    driver.execute_cdp_cmd("Emulation.setGeolocationOverride", GEO_DATA)
    driver.execute_cdp_cmd("Emulation.setUserAgentOverride", UA_Data)
    driver.execute_cdp_cmd("Emulation.setNavigatorOverrides",{"platform":"Linux armv8l"})
    
    return driver

def go_to_main_page(driver):
    """메인 페이지로 이동"""
    driver.get("https://www.myrealtrip.com/experiences/")
    time.sleep(random.uniform(CONFIG["MIN_DELAY"], CONFIG["MAX_DELAY"]))
    return True

def find_and_fill_search(driver, city_name):
    """검색창 찾기 및 입력"""
    print(f"  🔍 '{city_name}' 검색창 찾는 중...")
    search_selectors = [
        (By.CSS_SELECTOR, "input[placeholder*='어디로']"),
        (By.CSS_SELECTOR, "input[type='text']"),
        (By.XPATH, "//input[contains(@placeholder, '어디로')]"),
        (By.XPATH, "/html/body/main/div/div[2]/section[1]/div[1]/div/div/input")
    ]

    search_input = None
    for selector_type, selector_value in search_selectors:
        try:
            search_input = WebDriverWait(driver, CONFIG["WAIT_TIMEOUT"]).until(
                EC.presence_of_element_located((selector_type, selector_value))
            )
            print(f"  ✅ 검색창을 찾았습니다!")
            break
        except TimeoutException:
            continue

    if not search_input:
        raise NoSuchElementException("검색창을 찾을 수 없습니다")

    search_input.clear()
    search_input.send_keys(city_name)
    time.sleep(random.uniform(CONFIG["MIN_DELAY"], CONFIG["MIN_DELAY"]+2))
    print(f"  📝 '{city_name}' 키워드 입력 완료")
    return True

def click_search_button(driver):
    """검색 버튼 클릭"""
    print(f"  🔎 검색 버튼 찾는 중...")
    search_button_selectors = [
        (By.CSS_SELECTOR, "button[type='submit']"),
        (By.CSS_SELECTOR, ".search-btn"),
        (By.XPATH, "//button[contains(@class, 'search')]"),
        (By.XPATH, "//img[contains(@alt, '검색')]//parent::*"),
        (By.XPATH, "/html/body/main/div/div[2]/section[1]/div[1]/div/div/div/img")
    ]

    search_clicked = False
    for selector_type, selector_value in search_button_selectors:
        try:
            search_button = WebDriverWait(driver, CONFIG["WAIT_TIMEOUT"]).until(
                EC.element_to_be_clickable((selector_type, selector_value))
            )
            search_button.click()
            print(f"  ✅ 검색 버튼 클릭 성공!")
            search_clicked = True
            time.sleep(random.uniform(CONFIG["MIN_DELAY"], CONFIG["MAX_DELAY"]))
            break
        except TimeoutException:
            continue

    if not search_clicked:
        raise NoSuchElementException("검색 버튼을 찾을 수 없습니다")
    return True

def handle_popup(driver):
    """팝업 처리"""
    popup_selectors = [
        (By.CSS_SELECTOR, ".popup-close"),
        (By.CSS_SELECTOR, ".modal-close"),
        (By.XPATH, "//button[contains(@aria-label, '닫기')]"),
        (By.XPATH, "//button[contains(text(), '닫기')]"),
        (By.XPATH, "/html/body/div[15]/div[2]/button")
    ]

    popup_closed = False
    for selector_type, selector_value in popup_selectors:
        try:
            popup_button = WebDriverWait(driver, CONFIG["POPUP_WAIT"]).until(
                EC.element_to_be_clickable((selector_type, selector_value))
            )
            popup_button.click()
            print(f"  ✅ 팝업창을 닫았습니다.")
            popup_closed = True
            time.sleep(random.uniform(1, 4))
            break
        except TimeoutException:
            continue

    if not popup_closed:
        print(f"  ℹ️ 팝업창이 없거나 이미 닫혀있습니다.")
    return True

def click_view_all(driver):
    """전체 상품 보기 버튼 클릭"""
    print(f"  📋 전체 상품 보기 버튼 찾는 중...")
    view_all_selectors = [
        (By.XPATH, "//button[contains(text(), '전체')]"),
        (By.XPATH, "//span[contains(text(), '전체')]//parent::button"),
        (By.CSS_SELECTOR, "button[aria-label*='전체']"),
        (By.XPATH, "/html/body/div[4]/div[2]/div/div/div/span[21]/button")
    ]

    view_all_clicked = False
    for selector_type, selector_value in view_all_selectors:
        try:
            view_all_button = WebDriverWait(driver, CONFIG["WAIT_TIMEOUT"]).until(
                EC.element_to_be_clickable((selector_type, selector_value))
            )
            view_all_button.click()
            print(f"  ✅ 전체 상품 보기 클릭 성공!")
            view_all_clicked = True
            time.sleep(random.uniform(CONFIG["MIN_DELAY"], CONFIG["MIN_DELAY"]+3))
            break
        except TimeoutException:
            continue

    if not view_all_clicked:
        print(f"  ⚠️ 전체 상품 보기 버튼을 찾을 수 없습니다. 현재 상품으로 진행...")
    return True

def collect_page_urls(driver):
    """현재 페이지의 모든 상품 URL 수집"""
    print(f"  📊 현재 페이지의 상품 URL들을 수집 중...")
    
    time.sleep(random.uniform(3, 5))
    
    product_url_selectors = [
        "a[href*='/experiences/']",
        "a[href*='/experience/']",
        ".product-item a",
        ".experience-card a"
    ]
    
    collected_urls = []
    
    for selector in product_url_selectors:
        try:
            product_elements = driver.find_elements(By.CSS_SELECTOR, selector)
            
            for element in product_elements:
                try:
                    url = element.get_attribute('href')
                    if url and '/experiences/' in url and url not in collected_urls:
                        collected_urls.append(url)
                except Exception as e:
                    continue
            
            if collected_urls:
                break
                
        except Exception as e:
            continue
    
    valid_urls = []
    for url in collected_urls:
        if url and url.startswith('http') and '/experiences/' in url:
            valid_urls.append(url)
    
    print(f"  ✅ {len(valid_urls)}개의 상품 URL을 수집했습니다!")
    
    if len(valid_urls) == 0:
        print("  ⚠️ 상품 URL을 찾을 수 없습니다. 페이지 구조를 확인해주세요.")
    
    return valid_urls

def get_product_name(driver):
    """상품명 수집"""
    print(f"  📊 상품 정보 수집 중...")
    title_selectors = [
        (By.CSS_SELECTOR, "h1"),
        (By.CSS_SELECTOR, ".product-title"),
        (By.XPATH, "//h1[contains(@class, 'title')]"),
        (By.XPATH, "/html/body/div[1]/main/div[1]/section/div[1]/h1")
    ]

    for selector_type, selector_value in title_selectors:
        try:
            title_element = WebDriverWait(driver, CONFIG["WAIT_TIMEOUT"]).until(
                EC.presence_of_element_located((selector_type, selector_value))
            )
            found_name = title_element.text
            time.sleep(random.uniform(CONFIG["MIN_DELAY"], CONFIG["MIN_DELAY"]+2))
            return found_name
        except TimeoutException:
            continue
    
    raise NoSuchElementException("상품명을 찾을 수 없습니다")

def get_price(driver):
    """가격 정보 수집"""
    price_selectors = [
        (By.CSS_SELECTOR, ".price"),
        (By.CSS_SELECTOR, "[class*='price']"),
        (By.XPATH, "//span[contains(text(), '원')]"),
        (By.XPATH, "/html/body/div[1]/main/div[1]/div[4]/div/div/div[2]/span[2]")
    ]

    for selector_type, selector_value in price_selectors:
        try:
            price_element = WebDriverWait(driver, CONFIG["WAIT_TIMEOUT"]).until(
                EC.presence_of_element_located((selector_type, selector_value))
            )
            found_price = price_element.text
            time.sleep(random.uniform(CONFIG["MIN_DELAY"], CONFIG["MIN_DELAY"]+2))
            return found_price
        except TimeoutException:
            continue
    
    return "정보 없음"

def get_rating(driver):
    """평점 정보 수집"""
    rating_selectors = [
        (By.CSS_SELECTOR, ".rating"),
        (By.CSS_SELECTOR, "[class*='rating']"),
        (By.XPATH, "//span[contains(@class, 'rating')]"),
        (By.XPATH, "/html/body/div[1]/main/div[1]/section/div[1]/span/span[2]")
    ]

    for selector_type, selector_value in rating_selectors:
        try:
            rating_element = WebDriverWait(driver, CONFIG["WAIT_TIMEOUT"]).until(
                EC.presence_of_element_located((selector_type, selector_value))
            )
            found_rating = rating_element.text
            time.sleep(random.uniform(2, 4))
            return found_rating
        except TimeoutException:
            continue
    
    return "정보 없음"

def get_review_count(driver):
    """리뷰 수 정보 수집"""
    print(f"  📝 리뷰 수 정보 찾는 중...")
    review_count_selectors = [
        (By.XPATH, "//span[contains(text(), '리뷰')]"),
        (By.XPATH, "//span[contains(text(), 'review')]"),
        (By.XPATH, "//span[contains(text(), '후기')]"),
        (By.XPATH, "//span[contains(text(), '개')]"),
        (By.XPATH, "//span[contains(text(), '건')]"),
    ]

    for selector_type, selector_value in review_count_selectors:
        try:
            review_element = WebDriverWait(driver, 3).until(
                EC.presence_of_element_located((selector_type, selector_value))
            )
            review_text = review_element.text.strip()
            
            review_keywords = ['리뷰', '후기', 'review', '개', '건']
            has_number = any(char.isdigit() for char in review_text)
            has_keyword = any(keyword in review_text.lower() for keyword in review_keywords)
            
            if has_number and has_keyword and len(review_text) < 50:
                print(f"  ✅ 리뷰 수 정보 발견: {review_text}")
                return review_text
                
        except TimeoutException:
            continue

    print(f"  ℹ️ 리뷰 수 정보를 찾을 수 없습니다.")
    return ""

def get_language(driver):
    """언어 정보 수집"""
    print(f"  🌐 언어 정보 찾는 중...")
    language_selectors = [
        (By.XPATH, "//span[contains(text(), '언어')]"),
        (By.XPATH, "//span[contains(text(), '한국어')]"),
        (By.XPATH, "//span[contains(text(), '영어')]"),
        (By.XPATH, "//span[contains(text(), 'Korean')]"),
        (By.XPATH, "//span[contains(text(), 'English')]"),
    ]

    for selector_type, selector_value in language_selectors:
        try:
            language_element = WebDriverWait(driver, 3).until(
                EC.presence_of_element_located((selector_type, selector_value))
            )
            language_text = language_element.text.strip()
            
            language_keywords = ['언어', '한국어', '영어', '중국어', '일본어', 'Korean', 'English']
            
            if any(keyword in language_text for keyword in language_keywords):
                print(f"  ✅ 언어 정보 발견: {language_text}")
                return language_text
                
        except TimeoutException:
            continue

    print(f"  ℹ️ 언어 정보를 찾을 수 없습니다.")
    return ""

def download_image(driver, product_name, city_name):
    """이미지 다운로드"""
    if not CONFIG["SAVE_IMAGES"]:
        return "이미지 저장 비활성화"
        
    print(f"  🖼️ 상품 이미지 다운로드 중...")
    image_selectors = [
        (By.CSS_SELECTOR, ".product-image img"),
        (By.CSS_SELECTOR, ".gallery img:first-child"),
        (By.XPATH, "//img[contains(@alt, '상품')]"),
    ]

    img_url = None
    for selector_type, selector_value in image_selectors:
        try:
            img_element = WebDriverWait(driver, CONFIG["WAIT_TIMEOUT"]).until(
                EC.presence_of_element_located((selector_type, selector_value))
            )
            img_url = img_element.get_attribute('src')
            break
        except TimeoutException:
            continue

    if img_url:
        try:
            safe_product_name_raw = str(product_name) if product_name else "상품명없음"
            safe_product_name = make_safe_filename(f"{city_name}_{safe_product_name_raw[:CONFIG['MAX_PRODUCT_NAME_LENGTH']]}")
            img_download_path = os.path.abspath("") + "/myrealtripthumb_img/" + safe_product_name + ".png"
            urllib.request.urlretrieve(img_url, img_download_path)
            print(f"  ✅ 이미지 다운로드 완료!")
            return img_download_path
        except Exception as e:
            print(f"  ⚠️ 이미지 다운로드 실패: {type(e).__name__}: {e}")
            return "다운로드 실패"
    else:
        return "이미지 없음"

def crawl_all_products_in_page(driver, city_name, continent, country):
    """1페이지에서 설정된 개수만큼 상품을 순차적으로 크롤링하는 함수"""
    print(f"\n🎯 {city_name} - 1페이지에서 {CONFIG['MAX_PRODUCTS_PER_CITY']}개 상품 크롤링 시작!")
    
    # URL 수집
    try:
        product_urls = retry_operation(
            lambda: collect_page_urls(driver), 
            "상품 URL 수집"
        )
        
        if not product_urls:
            print(f"  ❌ {city_name}: 상품 URL을 찾을 수 없습니다.")
            return []
            
    except Exception as e:
        print(f"  ❌ {city_name}: URL 수집 실패 - {type(e).__name__}")
        return []
    
    # 설정된 개수만큼만 선택
    max_products = CONFIG["MAX_PRODUCTS_PER_CITY"]
    selected_urls = product_urls[:max_products]
    
    print(f"  📊 총 {len(product_urls)}개 상품 중 {len(selected_urls)}개를 크롤링합니다!")
    
    page_results = []
    total_products = len(selected_urls)
    
    print("  " + "="*60)
    
    for product_index, product_url in enumerate(selected_urls, 1):
        print_product_progress(product_index, total_products, f"상품 {product_index}")
        
        # 기본값 설정
        product_name = "정보 없음"
        price = "정보 없음"
        grade_review = "정보 없음"
        review_count = ""
        language = ""
        img_path = "처리 안됨"
        current_url = product_url
        
        try:
            # 상품 페이지로 이동
            print(f"    🔗 상품 {product_index} URL로 이동 중...")
            driver.get(product_url)
            time.sleep(random.uniform(CONFIG["MIN_DELAY"], CONFIG["MAX_DELAY"]))
            
            # 정보 수집
            try:
                product_name = retry_operation(
                    lambda: get_product_name(driver), 
                    f"상품 {product_index} 이름 수집"
                )
            except Exception as e:
                print(f"    ⚠️ 상품명 수집 실패: {type(e).__name__}")
                product_name = f"수집실패_{product_index}"

            try:
                price = get_price(driver)
            except Exception as e:
                print(f"    ⚠️ 가격 정보 수집 실패: {type(e).__name__}")
                price = "정보 없음"

            try:
                grade_review = get_rating(driver)
            except Exception as e:
                print(f"    ⚠️ 평점 정보 수집 실패: {type(e).__name__}")
                grade_review = "정보 없음"

            try:
                review_count = get_review_count(driver)
            except Exception as e:
                print(f"    ⚠️ 리뷰 수 정보 수집 실패: {type(e).__name__}")
                review_count = ""

            try:
                language = get_language(driver)
            except Exception as e:
                print(f"    ⚠️ 언어 정보 수집 실패: {type(e).__name__}")
                language = ""

            try:
                img_path = download_image(driver, product_name, city_name)
            except Exception as e:
                print(f"    ⚠️ 이미지 처리 실패: {type(e).__name__}")
                img_path = "처리 실패"

            # 결과 저장
            result = {
                '번호': len(page_results) + 1,
                '대륙': continent,
                '국가': country,
                '도시': city_name,
                '상품번호': product_index,
                '상품명': product_name,
                '가격': price,
                '평점': grade_review,
                '리뷰수': review_count,
                '언어': language,
                '이미지_경로': img_path,
                'URL': current_url,
                '수집_시간': time.strftime('%Y-%m-%d %H:%M:%S'),
                '상태': '완전수집'
            }

            page_results.append(result)

            # 상품 정보 출력
            safe_name = str(product_name)[:40] + "..." if len(str(product_name)) > 40 else str(product_name)
            print(f"    ✅ 상품 {product_index} 크롤링 완료!")
            print(f"       상품명: {safe_name}")
            print(f"       가격: {price}")
            print(f"       평점: {grade_review}")
            print(f"       리뷰수: {review_count if review_count else '정보 없음'}")
            print(f"       언어: {language if language else '정보 없음'}")
            
            # 다음 상품을 위한 휴식
            if product_index < total_products:
                wait_time = random.uniform(2, 5)
                print(f"    ⏰ 다음 상품까지 {wait_time:.1f}초 대기...")
                time.sleep(wait_time)

        except Exception as e:
            print(f"    ❌ 상품 {product_index}: 예상치 못한 오류 - {type(e).__name__}")
            continue
    
    print(f"\n  🎉 {city_name} - {len(selected_urls)}개 상품 크롤링 완료!")
    print(f"  📊 총 {len(page_results)}개 상품 정보를 수집했습니다.")
    print("  " + "="*60)
    
    return page_results

print("✅ 그룹 1 완료: 모든 함수가 정의되었습니다!")
print(f"🔢 현재 설정: {CONFIG['MAX_PRODUCTS_PER_CITY']}개 상품 크롤링")
print(f"🏙️ 검색 도시: {CITIES_TO_SEARCH}")
print("🎯 다음: 그룹 2를 실행하세요!")




🔧 Selenium 버전: 4.25.0
✅ 그룹 1 완료: 모든 함수가 정의되었습니다!
🔢 현재 설정: 2개 상품 크롤링
🏙️ 검색 도시: ['오사카']
🎯 다음: 그룹 2를 실행하세요!


In [ ]:
# 🚀 그룹 2: 도시 하드코딩 해결 - 드라이버 실행 + 도시 검색
# (이 셀을 실행하면 브라우저가 열리고 그룹 1에서 설정한 도시 검색까지 완료됩니다)

print("🔄 마이리얼트립 크롤링 시작!")

# 결과 저장소 초기화
all_results = []
print("🔄 결과 저장소 초기화 완료")

# 크롬 드라이버 실행
try:
    driver = setup_driver()
except Exception as e:
    print(f"❌ 드라이버 설정 실패: {e}")
    print("⚠️ 그룹 1을 먼저 실행했는지 확인하세요!")

# 썸네일 폴더 생성
if CONFIG["SAVE_IMAGES"]:
    img_folder_path = os.path.abspath("") + "/myrealtripthumb_img"
    try:
        shutil.rmtree(img_folder_path)
    except:
        pass
    os.makedirs(img_folder_path, exist_ok=True)
    print("📁 이미지 폴더 생성 완료")

# 🆕 그룹 1 설정에서 도시 가져오기
if not CITIES_TO_SEARCH:
    print("❌ CITIES_TO_SEARCH가 비어있습니다!")
    print("💡 그룹 1에서 CITIES_TO_SEARCH = ['도시명']을 설정하세요!")
    raise ValueError("검색할 도시가 설정되지 않았습니다")

city_name = CITIES_TO_SEARCH[0]  # 🆕 첫 번째 도시 사용
continent, country = get_city_info(city_name)

print("="*60)
print(f"🌏 그룹 1 설정 도시: {city_name} 검색 시작!")
print(f"⚙️  설정: 재시도 {CONFIG['RETRY_COUNT']}회, 타임아웃 {CONFIG['WAIT_TIMEOUT']}초")
print("="*60)

print(f"🔍 진행률: [░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░] 0.0% (0/1)")
print(f"📍 현재 작업: {city_name} - 진행중")
print("-" * 50)
print(f"  🌍 대륙: {continent} | 국가: {country}")

try:
    # 1. 메인 페이지로 이동
    print("  📱 마이리얼트립 메인 페이지 이동 중...")
    retry_operation(
        lambda: go_to_main_page(driver), 
        "메인 페이지 이동"
    )
    print(f"  ✅ 마이리얼트립 페이지 열기 완료")
    
    # 2. 검색창 찾기 및 입력 (🆕 동적 도시명 사용)
    retry_operation(
        lambda: find_and_fill_search(driver, city_name), 
        f"{city_name} 검색창 입력"
    )

    # 3. 검색 버튼 클릭
    retry_operation(
        lambda: click_search_button(driver), 
        "검색 버튼 클릭"
    )

    # 4. 팝업 처리 (선택사항)
    try:
        handle_popup(driver)
    except Exception as e:
        print(f"  ℹ️ 팝업 처리 중 오류 (무시됨): {type(e).__name__}")

    # 5. 전체 상품 보기 클릭 (선택사항)
    try:
        click_view_all(driver)
    except Exception as e:
        print(f"  ℹ️ 전체 상품 보기 처리 중 오류 (무시됨): {type(e).__name__}")

    print(f"\n✅ 그룹 2 완료: {city_name} 검색 성공!")
    print(f"🎯 현재 상태: {city_name} 상품 목록 페이지에 있습니다")
    print("🚀 다음: 그룹 3을 실행하여 URL 수집 테스트하세요!")
    
except TimeoutException as e:
    print(f"  ⏰ {city_name}: 페이지 로딩 시간 초과")
    print(f"  📍 위치: {continent} > {country} > {city_name}")
    print("❌ 그룹 2 실패: 시간 초과")
    
except NoSuchElementException as e:
    print(f"  🔍 {city_name}: 필요한 웹 요소를 찾을 수 없음")
    print(f"  📍 위치: {continent} > {country} > {city_name}")
    print("❌ 그룹 2 실패: 요소 없음")
    
except Exception as e:
    print(f"  ❌ {city_name}: 예상치 못한 오류 - {type(e).__name__}: {e}")
    print("❌ 그룹 2 실패: 예상치 못한 오류")
    print("💡 그룹 1을 먼저 실행했는지 확인하세요!")
    print(f"💡 CITIES_TO_SEARCH에 '{city_name}'이 올바르게 설정되었는지 확인하세요!")
    

🔄 마이리얼트립 크롤링 시작!
🔄 결과 저장소 초기화 완료
✅ 크롬 드라이버 실행 성공!
Windows
📁 이미지 폴더 생성 완료
🌏 그룹 1 설정 도시: 오사카 검색 시작!
⚙️  설정: 재시도 3회, 타임아웃 10초
🔍 진행률: [░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░] 0.0% (0/1)
📍 현재 작업: 오사카 - 진행중
--------------------------------------------------
  🌍 대륙: 아시아 | 국가: 일본
  📱 마이리얼트립 메인 페이지 이동 중...


KeyboardInterrupt: 

In [7]:
# 🔍 그룹 3: URL 수집 테스트 (24개 완전 수집 버전)
# (이 셀을 실행하면 현재 페이지에서 Products + Offers 24개 URL을 모두 수집합니다)

print("🔍 그룹 3: URL 수집 테스트 시작! (24개 완전 수집 버전)")
print("="*50)

def collect_all_24_urls(driver):
    print(f"  🔍 Products와 Offers URL 모두 수집 중...")
    
    # 페이지 로딩 완료 대기
    time.sleep(5)
    
    all_links = driver.find_elements(By.TAG_NAME, "a")
    print(f"  📊 전체 링크 수: {len(all_links)}개")
    
    products_urls = []
    offers_urls = []
    
    for link in all_links:
        try:
            href = link.get_attribute('href')
            if href:
                if 'experiences.myrealtrip.com/products/' in href:
                    if href not in products_urls:
                        products_urls.append(href)
                elif 'myrealtrip.com/offers/' in href:
                    if href not in offers_urls:
                        offers_urls.append(href)
        except:
            continue
    
    # 두 리스트 합치기 (products 먼저, offers 나중에)
    all_urls = products_urls + offers_urls
    
    print(f"  ✅ Products URL: {len(products_urls)}개")
    print(f"  ✅ Offers URL: {len(offers_urls)}개")
    print(f"  🎉 총 수집: {len(all_urls)}개")
    
    return all_urls

try:
    # 24개 URL 수집 실행
    print("📊 현재 페이지에서 24개 상품/오퍼 URL 수집을 시작합니다...")
    
    collected_urls = collect_all_24_urls(driver)
    
    if collected_urls:
        print(f"\n🎉 URL 수집 성공!")
        print(f"📈 총 {len(collected_urls)}개의 상품/오퍼 URL을 찾았습니다!")
        
        # 설정된 개수와 비교
        max_products = CONFIG['MAX_PRODUCTS_PER_CITY']
        will_crawl = min(len(collected_urls), max_products)
        
        print(f"🔢 설정된 크롤링 개수: {max_products}개")
        print(f"🎯 실제 크롤링할 개수: {will_crawl}개")
        
        print(f"\n📋 수집된 URL 목록:")
        print("-" * 80)
        for i, url in enumerate(collected_urls, 1):
            url_type = "🛍️ Product" if "/products/" in url else "🏷️ Offer"
            print(f"  {i:2d}. {url_type}: {url}")
        
        # URL 유효성 체크
        print(f"\n🔍 URL 유효성 체크:")
        products_count = sum(1 for url in collected_urls if '/products/' in url)
        offers_count = sum(1 for url in collected_urls if '/offers/' in url)
        
        print(f"  ✅ Products: {products_count}개")
        print(f"  ✅ Offers: {offers_count}개")
        print(f"  🎯 총 유효 URL: {products_count + offers_count}개")
        
        if len(collected_urls) >= 20:
            print("\n🚀 URL 수집 대성공! 다음 단계로 진행 가능합니다!")
            print("✅ 그룹 3 완료: 24개 URL 수집 테스트 성공")
            print("🎯 다음: 그룹 4를 실행하여 24개 상품/오퍼 크롤링을 시작하세요!")
        else:
            print("\n⚠️ 수집된 URL이 예상보다 적습니다.")
            
    else:
        print("❌ URL 수집 실패: 상품/오퍼 URL을 찾을 수 없습니다")
        print("💡 추가 디버깅:")
        
        # 디버깅 정보
        try:
            page_source = driver.page_source
            products_count = page_source.count('experiences.myrealtrip.com/products/')
            offers_count = page_source.count('myrealtrip.com/offers/')
            print(f"   📄 페이지 소스에서 products 패턴: {products_count}회")
            print(f"   📄 페이지 소스에서 offers 패턴: {offers_count}회")
            
        except Exception as debug_error:
            print(f"   ❌ 디버깅 실패: {debug_error}")
        
        print("🔄 그룹 2를 다시 실행해보세요!")
        
except NameError as e:
    print(f"❌ 변수 오류: {e}")
    print("💡 그룹 1과 그룹 2를 먼저 실행했는지 확인하세요!")
    
except Exception as e:
    print(f"❌ URL 수집 중 오류: {type(e).__name__}: {e}")
    print("💡 가능한 해결책:")
    print("   1. 그룹 2가 성공적으로 완료되었는지 확인")
    print("   2. 브라우저가 방콕 상품 목록 페이지에 있는지 확인")
    print("   3. 인터넷 연결 상태 확인")

print("\n" + "="*50)

🔍 그룹 3: URL 수집 테스트 시작! (24개 완전 수집 버전)
📊 현재 페이지에서 24개 상품/오퍼 URL 수집을 시작합니다...
  🔍 Products와 Offers URL 모두 수집 중...
  📊 전체 링크 수: 52개
  ✅ Products URL: 7개
  ✅ Offers URL: 17개
  🎉 총 수집: 24개

🎉 URL 수집 성공!
📈 총 24개의 상품/오퍼 URL을 찾았습니다!
🔢 설정된 크롤링 개수: 2개
🎯 실제 크롤링할 개수: 2개

📋 수집된 URL 목록:
--------------------------------------------------------------------------------
   1. 🛍️ Product: https://experiences.myrealtrip.com/products/3425896
   2. 🛍️ Product: https://experiences.myrealtrip.com/products/3417854
   3. 🛍️ Product: https://experiences.myrealtrip.com/products/3412140
   4. 🛍️ Product: https://experiences.myrealtrip.com/products/3154184
   5. 🛍️ Product: https://experiences.myrealtrip.com/products/3153597
   6. 🛍️ Product: https://experiences.myrealtrip.com/products/3153601
   7. 🛍️ Product: https://experiences.myrealtrip.com/products/3411998
   8. 🏷️ Offer: https://www.myrealtrip.com/offers/50211
   9. 🏷️ Offer: https://www.myrealtrip.com/offers/63685
  10. 🏷️ Offer: https://www.myrealtrip.com/

In [ ]:
# 🚀 하이브리드 구조 적용 - 수정된 그룹 4
# (그룹 1에서 모든 라이브러리 import 완료: re, json, datetime 포함)

# 도시 코드는 그룹 1에서 정의된 CITY_CODES 사용 (전 세계 도시 포함)


def get_city_code(city_name):
    """도시명으로 공항 코드 반환"""
    code = CITY_CODES.get(city_name, city_name[:3].upper())
    print(f"  🏙️ {city_name} → {code}")
    return code

def extract_clean_price(price_text):
    """가격 텍스트에서 실제 숫자 추출"""
    if not price_text or price_text == "정보 없음":
        return "정보 없음"
    
    price_patterns = [
        r'(\d{1,3}(?:,\d{3})*)\s*원[~-]?',  # "15,000원~" 형태
        r'(\d+)\s*원[~-]?',                 # "15000원~" 형태  
        r'(\d{1,3}(?:,\d{3})*)',           # 숫자만 (,포함)
    ]
    
    for pattern in price_patterns:
        match = re.search(pattern, price_text)
        if match:
            clean_price = match.group(1) + "원"
            print(f"    💰 가격 추출: '{price_text[:30]}...' → '{clean_price}'")
            return clean_price
    
    if any(keyword in price_text for keyword in ['할인', '쿠폰', '특가', '이벤트']):
        print(f"    🎫 할인정보: {price_text[:50]}...")
        return f"할인정보: {price_text}"
    
    print(f"    ⚠️ 가격 패턴 없음: {price_text[:30]}...")
    return price_text

def extract_clean_rating(rating_text):
    """평점 텍스트에서 숫자만 추출"""
    if not rating_text or rating_text == "정보 없음":
        return "정보 없음"
    
    rating_pattern = r'(\d+\.?\d*)'
    match = re.search(rating_pattern, rating_text)
    
    if match:
        clean_rating = match.group(1)
        print(f"    ⭐ 평점 추출: '{rating_text}' → '{clean_rating}'")
        return clean_rating
    
    return rating_text

def create_hierarchical_image_path(continent, country, city, filename):
    """대륙-국가-도시 계층 구조로 이미지 경로 생성"""
    # 마이리얼트립 전용 기본 경로 (현재 폴더 기준)
    base_folder = "myrealtripthumb_img"  # myrealtrip/ 제거
    
    # 계층 구조 경로 생성
    hierarchical_path = os.path.join(base_folder, continent, country, city)
    
    # 전체 파일 경로
    full_path = os.path.join(hierarchical_path, filename)
    
    print(f"  📁 계층 경로: {hierarchical_path}")
    print(f"  📷 파일 경로: {full_path}")
    
    return hierarchical_path, full_path

def create_image_filename(city_name, product_index):
    """공항 코드 + 순서 기반 이미지 파일명 생성"""
    city_code = get_city_code(city_name)
    filename = f"{city_code}_{product_index:03d}.jpg"
    print(f"  📷 이미지 파일명: {filename}")
    return filename

def get_price_fixed(driver):
    """마이리얼트립 실제 가격 수집 (수정된 버전)"""
    print(f"  💰 가격 정보 수집 중...")
    
    price_selectors = [
        (By.CSS_SELECTOR, "span[style*='color: rgb(255, 71, 71)']"),
        (By.CSS_SELECTOR, "span[style*='color:#ff4747']"),
        (By.CSS_SELECTOR, ".text-red"),
        (By.XPATH, "//span[contains(text(), '원~') and not(contains(text(), '쿠폰')) and not(contains(text(), '할인')) and not(contains(text(), '받기'))]"),
        (By.XPATH, "//span[contains(text(), '원-') and not(contains(text(), '쿠폰')) and not(contains(text(), '할인')) and not(contains(text(), '받기'))]"),
        (By.XPATH, "//span[contains(text(), ',') and contains(text(), '원') and not(contains(text(), '쿠폰')) and not(contains(text(), '할인')) and not(contains(text(), '받기'))]"),
        (By.CSS_SELECTOR, ".price"),
        (By.CSS_SELECTOR, "[class*='price']"),
    ]

    for selector_type, selector_value in price_selectors:
        try:
            price_elements = driver.find_elements(selector_type, selector_value)
            
            for element in price_elements:
                try:
                    price_text = element.text.strip()
                    
                    if (price_text and 
                        '원' in price_text and 
                        any(char.isdigit() for char in price_text) and
                        not any(keyword in price_text for keyword in ['쿠폰', '할인', '받기', '다운', '적립', '포인트', '혜택', '클릭'])):
                        
                        print(f"  ✅ 가격 발견: {price_text}")
                        return price_text
                        
                except Exception:
                    continue
                    
        except Exception:
            continue
    
    print(f"  ⚠️ 가격 정보를 찾을 수 없습니다")
    return "정보 없음"

def get_main_product_image_url(driver):
    """대표 상품 이미지 URL 찾기 (수정된 버전)"""
    print(f"  🖼️ 대표 이미지 URL 찾는 중...")
    
    image_selectors = [
        (By.CSS_SELECTOR, ".main-image img"),
        (By.CSS_SELECTOR, ".product-main-image img"),
        (By.CSS_SELECTOR, ".hero-image img"),
        (By.CSS_SELECTOR, ".gallery img:first-child"),
        (By.CSS_SELECTOR, ".image-gallery img:first-child"),
        (By.CSS_SELECTOR, ".product-gallery img:first-child"),
        (By.XPATH, "//img[@width and @height and (@width > '300' or @height > '300')]"),
        (By.CSS_SELECTOR, "img[src*='cdn'][src*='large']"),
        (By.CSS_SELECTOR, "img[src*='cdn'][src*='main']"),
        (By.CSS_SELECTOR, ".product-image img"),
        (By.CSS_SELECTOR, "img[src*='http']"),
    ]

    for selector_type, selector_value in image_selectors:
        try:
            img_elements = driver.find_elements(selector_type, selector_value)
            
            for img_element in img_elements:
                try:
                    img_url = img_element.get_attribute('src')
                    
                    if img_url and img_url.startswith('http'):
                        try:
                            width = img_element.get_attribute('width') or img_element.size.get('width', 0)
                            height = img_element.get_attribute('height') or img_element.size.get('height', 0)
                            
                            if width and height and (int(str(width).replace('px','')) < 100 or int(str(height).replace('px','')) < 100):
                                continue
                                
                        except:
                            pass
                        
                        exclude_patterns = ['logo', 'icon', 'banner', 'ad', 'avatar', 'profile']
                        if any(pattern in img_url.lower() for pattern in exclude_patterns):
                            continue
                        
                        print(f"  ✅ 대표 이미지 URL 발견: {img_url[:60]}...")
                        return img_url
                        
                except Exception:
                    continue
                    
        except Exception:
            continue
    
    print(f"  ⚠️ 대표 이미지 URL을 찾을 수 없습니다")
    return None

def download_image_hybrid(driver, product_name, continent, country, city_name, product_index):
    """하이브리드 구조용 이미지 다운로드"""
    
    if not CONFIG["SAVE_IMAGES"]:
        return {
            'status': '이미지 저장 비활성화',
            'filename': None,
            'filepath': None,
            'filesize': 0
        }
    
    print(f"  🖼️ 하이브리드 구조로 이미지 다운로드 시작...")
    
    img_url = get_main_product_image_url(driver)
    
    if not img_url:
        return {
            'status': '이미지 URL 없음',
            'filename': None,
            'filepath': None,
            'filesize': 0
        }

    try:
        # 파일명 생성
        filename = create_image_filename(city_name, product_index)
        
        # 하이브리드 계층 구조 경로 생성
        hierarchical_path, full_path = create_hierarchical_image_path(
            continent, country, city_name, filename
        )
        
        # 폴더 생성 (계층 구조)
        os.makedirs(hierarchical_path, exist_ok=True)
        print(f"  ✅ 계층 폴더 생성 완료: {hierarchical_path}")
        
        # 이미지 다운로드
        import urllib.request
        urllib.request.urlretrieve(img_url, full_path)
        
        # 파일 검증
        if os.path.exists(full_path):
            file_size = os.path.getsize(full_path)
            if file_size > 1000:  # 1KB 이상이어야 유효한 이미지
                print(f"  ✅ 하이브리드 구조 이미지 저장 완료! (크기: {file_size:,} bytes)")
                return {
                    'status': '다운로드 완료',
                    'filename': filename,
                    'filepath': full_path,
                    'relative_path': f"{continent}/{country}/{city_name}/{filename}",
                    'filesize': file_size,
                    'original_url': img_url
                }
            else:
                print(f"  ❌ 이미지 파일이 너무 작습니다 ({file_size} bytes)")
                os.remove(full_path)
                return {
                    'status': '파일 너무 작음',
                    'filename': None,
                    'filepath': None,
                    'filesize': 0
                }
        else:
            print(f"  ❌ 이미지 파일 생성 실패")
            return {
                'status': '파일 생성 실패',
                'filename': None,
                'filepath': None,
                'filesize': 0
            }
            
    except Exception as e:
        print(f"  ⚠️ 이미지 다운로드 실패: {type(e).__name__}: {e}")
        return {
            'status': f'다운로드 실패: {type(e).__name__}',
            'filename': None,
            'filepath': None,
            'filesize': 0
        }

def save_myrealtrip_data(products_data):
    """마이리얼트립 데이터를 하이브리드 구조로 저장"""
    print("💾 하이브리드 구조로 데이터 저장 중...")
    
    # 현재 폴더 사용 (이미 myrealtrip 폴더 안이므로)
    mrt_folder = "."  # 현재 디렉토리
    
    # 사이트 정보 추가
    for product in products_data:
        product['사이트'] = "마이리얼트립"
        product['사이트_코드'] = "MRT"
        product['업데이트_시간'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    
    # 개별 CSV 저장 (현재 폴더에)
    df = pd.DataFrame(products_data)
    csv_path = "myrealtrip_products.csv"  # 현재 폴더에 직접 저장
    df.to_csv(csv_path, index=False, encoding='utf-8-sig')
    
    print(f"✅ 마이리얼트립 개별 CSV 저장: {csv_path}")
    print(f"📊 저장된 상품 수: {len(products_data)}개")
    
    # 메타데이터 업데이트
    metadata = {
        "myrealtrip": {
            "last_update": datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            "product_count": len(products_data),
            "status": "success",
            "csv_path": csv_path,
            "image_path": "myrealtripthumb_img"  # 경로 수정
        }
    }
    
    with open("data_metadata.json", 'w', encoding='utf-8') as f:
        json.dump(metadata, f, ensure_ascii=False, indent=2)
    
    print(f"✅ 메타데이터 업데이트 완료: data_metadata.json")
    
    return csv_path

# === 메인 크롤링 코드 (하이브리드 구조) ===

print("🚀 하이브리드 구조 그룹 4: 도시별 크롤링!")
print("="*80)

# 🆕 그룹 1 설정에서 도시 가져오기  
if not CITIES_TO_SEARCH:
    print("❌ CITIES_TO_SEARCH가 비어있습니다!")
    print("💡 그룹 1에서 CITIES_TO_SEARCH = ['도시명']을 설정하세요!")
    raise ValueError("검색할 도시가 설정되지 않았습니다")

city_name = CITIES_TO_SEARCH[0]  # 🆕 첫 번째 도시 사용
continent, country = get_city_info(city_name)

urls_to_crawl = collected_urls[:CONFIG['MAX_PRODUCTS_PER_CITY']]
total_products = len(urls_to_crawl)

print(f"📊 그룹 1 설정값: {CONFIG['MAX_PRODUCTS_PER_CITY']}개 상품")
print(f"🎯 실제 크롤링 대상: {total_products}개 상품")
print(f"🌍 위치: {continent} > {country} > {city_name}")
print(f"✈️ 공항 코드: {get_city_code(city_name)}")
print(f"📁 이미지 저장: myrealtripthumb_img/{continent}/{country}/{city_name}/")

# 크롤링할 URL 목록 출력
print("\n📋 크롤링할 상품 목록:")
for i, url in enumerate(urls_to_crawl, 1):
    url_type = "🛍️ Product" if "/products/" in url else "🏷️ Offer"
    image_name = create_image_filename(city_name, i)
    print(f"  {i}. {url_type}: {url}")
    print(f"     → 이미지: {continent}/{country}/{city_name}/{image_name}")

print("="*80)

# 결과 저장 리스트
all_results = []

# 각 상품별 크롤링
for product_index, product_url in enumerate(urls_to_crawl, 1):
    print_product_progress(product_index, total_products, f"상품 {product_index}")
    
    # 기본값 설정
    product_name = "정보 없음"
    price_raw = "정보 없음"
    price_clean = "정보 없음"
    rating_raw = "정보 없음"
    rating_clean = "정보 없음"
    review_count = ""
    language = ""
    img_info = {'status': '처리 안됨', 'filename': None, 'filepath': None}
    url_type = "Product" if "/products/" in product_url else "Offer"
    
    try:
        # 🔗 개별 상품 페이지로 이동
        print(f"    🔗 상품 {product_index} URL로 이동 중...")
        print(f"    📍 {url_type}: {product_url}")
        
        driver.get(product_url)
        time.sleep(random.uniform(CONFIG["MIN_DELAY"], CONFIG["MAX_DELAY"]))
        
        if driver.current_url != product_url:
            print(f"    ⚠️ URL 변경됨: {driver.current_url}")
        else:
            print(f"    ✅ 정상 이동 완료")
        
        # 상품 정보 수집
        try:
            product_name = get_product_name(driver)
            print(f"    ✅ 상품명 수집 완료")
        except Exception as e:
            print(f"    ⚠️ 상품명 수집 실패: {type(e).__name__}")
            product_name = f"수집실패_{product_index}"

        try:
            price_raw = get_price_fixed(driver)
            price_clean = extract_clean_price(price_raw)
            print(f"    ✅ 가격 수집 완료")
        except Exception as e:
            print(f"    ⚠️ 가격 수집 실패: {type(e).__name__}")

        try:
            rating_raw = get_rating(driver)
            rating_clean = extract_clean_rating(rating_raw)
            print(f"    ✅ 평점 수집 완료")
        except Exception as e:
            print(f"    ⚠️ 평점 수집 실패: {type(e).__name__}")

        try:
            review_count = get_review_count(driver)
            if review_count:
                print(f"    ✅ 리뷰수 수집: {review_count}")
        except Exception as e:
            print(f"    ⚠️ 리뷰수 수집 실패: {type(e).__name__}")

        try:
            language = get_language(driver)
            if language:
                print(f"    ✅ 언어 수집: {language}")
        except Exception as e:
            print(f"    ⚠️ 언어 수집 실패: {type(e).__name__}")

        try:
            img_info = download_image_hybrid(driver, product_name, continent, country, city_name, product_index)
        except Exception as e:
            print(f"    ⚠️ 이미지 처리 실패: {type(e).__name__}")

        # 결과 저장 (하이브리드 구조)
        result = {
            '번호': len(all_results) + 1,
            '대륙': continent,
            '국가': country,
            '도시': city_name,
            '공항코드': get_city_code(city_name),
            '상품번호': product_index,
            '상품타입': url_type,
            '상품명': product_name,
            '가격_원본': price_raw,
            '가격_정제': price_clean,
            '평점_원본': rating_raw,
            '평점_정제': rating_clean,
            '리뷰수': review_count,
            '언어': language,
            '이미지_파일명': img_info.get('filename', ''),
            '이미지_상대경로': img_info.get('relative_path', ''),
            '이미지_전체경로': img_info.get('filepath', ''),
            '이미지_상태': img_info.get('status', ''),
            '이미지_크기': img_info.get('filesize', 0),
            'URL': product_url,
            '수집_시간': time.strftime('%Y-%m-%d %H:%M:%S'),
            '상태': '완전수집'
        }
        
        all_results.append(result)

        # 상품 정보 요약 출력
        safe_name = str(product_name)[:40] + "..." if len(str(product_name)) > 40 else str(product_name)
        print(f"    🎉 상품 {product_index} 완료!")
        print(f"       타입: {url_type}")
        print(f"       상품명: {safe_name}")
        print(f"       가격: {price_clean}")
        print(f"       평점: {rating_clean}")
        print(f"       이미지: {img_info.get('relative_path', '없음')}")
        if review_count:
            print(f"       리뷰수: {review_count}")
        if language:
            print(f"       언어: {language}")
        
        # 다음 상품을 위한 휴식
        if product_index < total_products:
            wait_time = random.uniform(2, 4)
            print(f"    ⏰ 다음 상품까지 {wait_time:.1f}초 대기...")
            time.sleep(wait_time)

    except Exception as e:
        print(f"    ❌ 상품 {product_index} 오류: {type(e).__name__}: {e}")
        # 오류 시 기본 정보 저장
        result = {
            '번호': len(all_results) + 1,
            '대륙': continent,
            '국가': country, 
            '도시': city_name,
            '공항코드': get_city_code(city_name),
            '상품번호': product_index,
            '상품타입': url_type,
            '상품명': f"오류_{product_index}",
            '가격_원본': "수집실패",
            '가격_정제': "수집실패",
            '평점_원본': "수집실패",
            '평점_정제': "수집실패", 
            '리뷰수': "",
            '언어': "",
            '이미지_파일명': "",
            '이미지_상대경로': "",
            '이미지_전체경로': "",
            '이미지_상태': "처리실패",
            '이미지_크기': 0,
            'URL': product_url,
            '수집_시간': time.strftime('%Y-%m-%d %H:%M:%S'),
            '상태': f'오류({type(e).__name__})'
        }
        all_results.append(result)
        continue

# 최종 결과 처리
print(f"\n🎉 하이브리드 구조 크롤링 완료!")
print(f"📊 수집 결과: {len(all_results)}개 상품")

# 타입별 통계
product_count = len([r for r in all_results if r['상품타입'] == 'Product'])
offer_count = len([r for r in all_results if r['상품타입'] == 'Offer']) 
success_count = len([r for r in all_results if r['상태'] == '완전수집'])
image_success = len([r for r in all_results if r['이미지_상태'] == '다운로드 완료'])

print(f"🛍️ Products: {product_count}개")
print(f"🏷️ Offers: {offer_count}개")
print(f"✅ 크롤링 성공: {success_count}개")
print(f"📷 이미지 성공: {image_success}개")
print(f"📊 전체 성공률: {success_count/len(all_results)*100:.1f}%")
print(f"🖼️ 이미지 성공률: {image_success/len(all_results)*100:.1f}%")

# 상품별 간단 요약
print(f"\n📋 수집된 상품 요약:")
for i, result in enumerate(all_results, 1):
    status_emoji = "✅" if result['상태'] == '완전수집' else "❌"
    img_emoji = "📷" if result['이미지_상태'] == '다운로드 완료' else "🚫"
    print(f"  {i}. {status_emoji}{img_emoji} [{result['상품타입']}] {result['상품명'][:30]}...")
    print(f"      가격: {result['가격_정제']} | 평점: {result['평점_정제']}")
    if result['이미지_상대경로']:
        print(f"      이미지: {result['이미지_상대경로']}")

# 하이브리드 구조로 저장
if all_results:
    try:
        saved_path = save_myrealtrip_data(all_results)
        
        print(f"\n🏆 하이브리드 구조 저장 완료!")
        print(f"📁 개별 CSV: {saved_path}")
        print(f"📁 이미지 폴더: myrealtripthumb_img/{continent}/{country}/{city_name}/")
        print(f"📁 메타데이터: data_metadata.json")
        
        # 폴더 구조 확인
        img_folder = f"myrealtripthumb_img/{continent}/{country}/{city_name}"
        if os.path.exists(img_folder):
            img_files = os.listdir(img_folder)
            print(f"\n📷 저장된 이미지: {len(img_files)}개")
            for img_file in img_files:
                file_size = os.path.getsize(os.path.join(img_folder, img_file))
                print(f"     📸 {img_file} ({file_size:,} bytes)")
        
    except Exception as e:
        print(f"❌ 하이브리드 저장 실패: {e}")

print(f"\n🎯 하이브리드 구조 마이리얼트립 크롤링 완료!")
print(f"🏗️ 구조: 개별 관리 + 계층 이미지 저장")
print(f"🚀 다음: 클룩, KKday도 같은 구조로 확장 가능!")
print(f"💡 통합 분석: ProductDataManager로 전체 사이트 비교 분석!")

🚀 하이브리드 구조 그룹 4: 도시별 크롤링!
📊 그룹 1 설정값: 2개 상품
🎯 실제 크롤링 대상: 2개 상품
🌍 위치: 아시아 > 일본 > 오사카
  🏙️ 오사카 → KIX
✈️ 공항 코드: KIX
📁 이미지 저장: myrealtripthumb_img/아시아/일본/오사카/

📋 크롤링할 상품 목록:
  🏙️ 오사카 → KIX
  📷 이미지 파일명: KIX_001.jpg
  1. 🛍️ Product: https://experiences.myrealtrip.com/products/3425896
     → 이미지: 아시아/일본/오사카/KIX_001.jpg
  🏙️ 오사카 → KIX
  📷 이미지 파일명: KIX_002.jpg
  2. 🛍️ Product: https://experiences.myrealtrip.com/products/3417854
     → 이미지: 아시아/일본/오사카/KIX_002.jpg
    🎯 상품 진행률: [██████████░░░░░░░░░░] 50.0% (1/2)
    📦 현재 상품: 상품 1
    🔗 상품 1 URL로 이동 중...
    📍 Product: https://experiences.myrealtrip.com/products/3425896
    ✅ 정상 이동 완료
  📊 상품 정보 수집 중...
    ✅ 상품명 수집 완료
  💰 가격 정보 수집 중...
  ✅ 가격 발견: 49,000원~
    💰 가격 추출: '49,000원~...' → '49,000원'
    ✅ 가격 수집 완료
    ⭐ 평점 추출: '4.9 ·' → '4.9'
    ✅ 평점 수집 완료
  📝 리뷰 수 정보 찾는 중...
  ✅ 리뷰 수 정보 발견: 후기 2,390개
    ✅ 리뷰수 수집: 후기 2,390개
  🌐 언어 정보 찾는 중...
  ✅ 언어 정보 발견: 한국어
    ✅ 언어 수집: 한국어
  🖼️ 하이브리드 구조로 이미지 다운로드 시작...
  🖼️ 대표 이미지 URL 찾는 중...
  ✅ 대표 이미지 URL 발견: htt